In [12]:
import requests, zipfile, io
from datasets import load_dataset

In [13]:
from torch.utils.data import Dataset
import os
from PIL import Image

class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, root_dir, feature_extractor, train=True):
        """
        Args:
            root_dir (string): Root directory of the dataset containing the images + annotations.
            feature_extractor (SegFormerFeatureExtractor): feature extractor to prepare images + segmentation maps.
            train (bool): Whether to load "training" or "validation" images + annotations.
        """
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.train = train

        sub_path = "train" if self.train else "validation"
        self.img_dir = os.path.join(self.root_dir, sub_path, "rgb")
        self.ann_dir = os.path.join(self.root_dir, sub_path, "labels")
        
        # read images
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
          image_file_names.extend(files)
        self.images = sorted(image_file_names)
        
        # read annotations
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
          annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        assert len(self.images) == len(self.annotations), "There must be as many images as there are segmentation maps"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = Image.open(os.path.join(self.img_dir, self.images[idx]))
        segmentation_map = Image.open(os.path.join(self.ann_dir, self.annotations[idx]))

        # randomly crop + pad both image and segmentation map to same size
        encoded_inputs = self.feature_extractor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
          encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs

In [32]:
from transformers import SegformerFeatureExtractor

root_dir = '/home/klimenko/seg_materials/VAL_SEGFORMER/data/4/'# '/home/klimenko/facade_materials/materials/'
feature_extractor = SegformerFeatureExtractor(reduce_labels=True)

train_dataset = SemanticSegmentationDataset(root_dir=root_dir, feature_extractor=feature_extractor)
valid_dataset = SemanticSegmentationDataset(root_dir=root_dir, feature_extractor=feature_extractor, train=False)

/home/klimenko/anaconda3/envs/facade/lib/python3.10/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/klimenko/anaconda3/envs/facade/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [33]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))

Number of training examples: 121
Number of validation examples: 23


In [34]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=1)

In [35]:
from transformers import SegformerForSemanticSegmentation
import json
from huggingface_hub import cached_download, hf_hub_url, hf_hub_download

# load id2label mapping from a JSON on the hub
id2label = json.load(open('materials.json'))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

# define model
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0",
                                                         num_labels=7, 
                                                         id2label=id2label, 
                                                         label2id=label2id,
)


/home/klimenko/anaconda3/envs/facade/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/klimenko/anaconda3/envs/facade/lib/python3.10/site-packages/transformers/modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are ex

In [36]:
from datasets import load_metric
metric = load_metric("mean_iou")

In [37]:
import torch
import numpy as np
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for epoch in range(12):  # loop over the dataset multiple times
    print("Epoch:", epoch)
    loss_list = []
    model.train()

    
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs;
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)
        
        #print(pixel_values.shape)
        #print(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        print(idx)
        
        loss.backward()
        optimizer.step()

        # evaluate
#         with torch.no_grad():
#             upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
#             predicted = upsampled_logits.argmax(dim=1)
#             metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())
#             metrics = metric.compute(num_labels=len(id2label), ignore_index=255,reduce_labels=False)
            
#             if idx % 1 == 0:
#                 print("idx: ", str(idx)," Loss:", str(loss.item())[0:7]," Mean_iou:", str(metrics["mean_iou"])[0:5], " Mean accuracy:", str(metrics["mean_accuracy"])[0:5] )
#         loss_list.append(loss.item())
        
#     loss_value = np.mean(np.array(loss_list))  
#     print('loss mean ', loss_value)
    #model.save_pretrained("weights/fold_0_"+str(epoch)+"_ep_"+str(loss_value)[0:4]+".pth")
    
    model.eval()
    
    for idx, batch in enumerate(tqdm(valid_dataloader)):
        # get the inputs;
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)
        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        
#         loss.backward()
#         optimizer.step()

        # evaluate
        with torch.no_grad():
            upsampled_logits = nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
            predicted = upsampled_logits.argmax(dim=1)
            metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())
            metrics = metric.compute(num_labels=len(id2label), ignore_index=255,reduce_labels=False)
            
            if idx % 1 == 0:
                print("idx: ", str(idx)," Loss:", str(loss.item())[0:7]," Mean_iou:", str(metrics["mean_iou"])[0:5], " Mean accuracy:", str(metrics["mean_accuracy"])[0:5] )
        loss_list.append(loss.item())
        
    loss_value = np.mean(np.array(loss_list))  
    print('loss mean ', loss_value)
    model.save_pretrained("weights/fold_4_"+str(epoch)+"_ep_"+str(loss_value)[0:4]+".pth")

Epoch: 0


  0%|          | 0/13 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12


  0%|          | 0/23 [00:00<?, ?it/s]

/home/klimenko/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/927b58f57da3f4b6e385e47d8a4b3947ee3f7cfcdba9b9359eba2ada2ed6b951/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


...

idx:  0  Loss: 1.06734  Mean_iou: 0.159  Mean accuracy: 0.198
idx:  1  Loss: 0.45372  Mean_iou: 0.422  Mean accuracy: 0.480
idx:  2  Loss: 0.57205  Mean_iou: 0.375  Mean accuracy: 0.460
idx:  3  Loss: 0.51124  Mean_iou: 0.277  Mean accuracy: 0.313
idx:  4  Loss: 1.15408  Mean_iou: 0.192  Mean accuracy: 0.310
idx:  5  Loss: 1.07283  Mean_iou: 0.180  Mean accuracy: 0.249
idx:  6  Loss: 0.71384  Mean_iou: 0.287  Mean accuracy: 0.350
idx:  7  Loss: 0.64581  Mean_iou: 0.315  Mean accuracy: 0.400
idx:  8  Loss: 0.85248  Mean_iou: 0.333  Mean accuracy: 0.447
idx:  9  Loss: 1.16473  Mean_iou: 0.146  Mean accuracy: 0.216
idx:  10  Loss: 1.74269  Mean_iou: 0.051  Mean accuracy: 0.253
idx:  11  Loss: 1.16751  Mean_iou: 0.167  Mean accuracy: 0.215
idx:  12  Loss: 0.65463  Mean_iou: 0.260  Mean accuracy: 0.319
idx:  13  Loss: 0.96731  Mean_iou: 0.242  Mean accuracy: 0.305
idx:  14  Loss: 1.25396  Mean_iou: 0.104  Mean accuracy: 0.174
idx:  15  Loss: 1.03438  Mean_iou: 0.204  Mean accuracy: 0.294
id

  0%|          | 0/13 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12


  0%|          | 0/23 [00:00<?, ?it/s]

idx:  0  Loss: 0.58644  Mean_iou: 0.264  Mean accuracy: 0.290
idx:  1  Loss: 0.54912  Mean_iou: 0.401  Mean accuracy: 0.471
idx:  2  Loss: 0.40303  Mean_iou: 0.392  Mean accuracy: 0.436
idx:  3  Loss: 0.39193  Mean_iou: 0.261  Mean accuracy: 0.289
idx:  4  Loss: 0.97780  Mean_iou: 0.177  Mean accuracy: 0.278
idx:  5  Loss: 0.82842  Mean_iou: 0.195  Mean accuracy: 0.254
idx:  6  Loss: 0.68324  Mean_iou: 0.344  Mean accuracy: 0.431
idx:  7  Loss: 0.41664  Mean_iou: 0.342  Mean accuracy: 0.375
idx:  8  Loss: 0.72406  Mean_iou: 0.334  Mean accuracy: 0.449
idx:  9  Loss: 1.24964  Mean_iou: 0.139  Mean accuracy: 0.214
idx:  10  Loss: 1.67485  Mean_iou: 0.085  Mean accuracy: 0.256
idx:  11  Loss: 0.94927  Mean_iou: 0.199  Mean accuracy: 0.232
idx:  12  Loss: 0.57486  Mean_iou: 0.260  Mean accuracy: 0.320
idx:  13  Loss: 0.88788  Mean_iou: 0.217  Mean accuracy: 0.288
idx:  14  Loss: 1.17218  Mean_iou: 0.112  Mean accuracy: 0.189
idx:  15  Loss: 0.95373  Mean_iou: 0.248  Mean accuracy: 0.328
id

In [ ]:
model.save_pretrained("fold_0_1ep.pth")

In [ ]:
model = SegformerForSemanticSegmentation.from_pretrained("fold_0_5ep.pth")
model.to(device)

In [ ]:

model = SegformerForSemanticSegmentation.from_pretrained("fold_0_5ep.pth")
model.to(device)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


for filename in os.listdir(root_dir+'validation/rgb/'):
    
    image_path = root_dir+'validation/rgb/'+filename
    image = Image.open(image_path).convert("RGB")

    inputs = feature_extractor(images=image, return_tensors="pt").to(device)
    resized_img = image.resize((128, 128))
    image_np = np.array(resized_img)
    
    model.eval()
    with torch.no_grad():
        outputs = model(inputs.pixel_values).logits
        upsampled_logits = nn.functional.interpolate(outputs,size=image.size[::-1],mode='bilinear',align_corners=False)
        print(upsampled_logits.shape)
        seg = upsampled_logits.cpu().argmax(dim=1)[0].numpy()
        replacement_dict = {0: 10, 1: 11, 2:12, 3:13, 4:16, 5:17, 6:0}
        seg2 = np.vectorize(replacement_dict.get)(seg)
        seg3 = np.stack([seg2] * 3, axis=-1)
        
        np.save('/home/klimenko/seg_materials/VAL_SEGFORMER/results/'+filename.replace(".png", ".npy"), seg2)
        
        
        #image = Image.fromarray(seg3.astype(np.uint8))
        #image.save('/home/klimenko/seg_materials/VAL_SEGFORMER/results/'+filename.replace(".png", ".jpg"))
    
    




In [ ]:
image_path = "/home/klimenko/seg_materials/VAL_SEGFORMER/data/0/train/rgb/Copy of YAqsjUKCMc_DkjpPMtrIcQ_180.png"  # replace with your image path
image = Image.open(image_path).convert("RGB")

inputs = feature_extractor(images=image, return_tensors="pt").to(device)
resized_img = image.resize((128, 128))
image_np = np.array(resized_img)

In [ ]:
image.size

In [ ]:

upsampled_logits = nn.functional.interpolate(outputs,size=(1000, 1000)[::-1],mode='bilinear',align_corners=False)

In [ ]:
upsampled_logits.shape

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(inputs.pixel_values).logits
    upsampled_logits = nn.functional.interpolate(outputs,size=(1000, 1000)[::-1],mode='bilinear',align_corners=False)
    print(upsampled_logits.shape)
    seg = upsampled_logits.cpu().argmax(dim=1)[0].numpy()

In [ ]:
seg.shape

In [ ]:
import numpy as np
outputs_np = upsampled_logits.cpu().numpy()[0]

In [ ]:
{
  "0": "nothing",
  "1": "10_glazing",
  "2": "11_concrete",
  "3": "12_masonry",
  "4": "13_siding",
  "5": "16_stucco",
  "6": "17_metal"
}




In [ ]:
replacement_dict = {0: 10, 1: 11, 2:12, 3:13, 4:16, 5:17, 6:0}
seg2 = np.vectorize(replacement_dict.get)(seg)
seg3 = np.stack([seg2] * 3, axis=-1)

In [ ]:
seg2

In [ ]:
import cv2
image_path = "/home/klimenko/seg_materials/VAL_SEGFORMER/data/0/train/labels/Copy of YAqsjUKCMc_DkjpPMtrIcQ_180.png"
qq = cv2.imread(image_path)

In [ ]:
%matplotlib notebook
plt.imshow(qq*10)

In [ ]:
seg = outputs.logits.cpu().argmax(dim=1)[0].numpy()

In [ ]:
print(np.unique(seg))